In [1]:
# set cuda visible devices
def is_notebook() -> bool:
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter

import os
if is_notebook():
    os.environ["CUDA_VISIBLE_DEVICES"] = "" #"1"
    # os.environ['CUDA_LAUNCH_BLOCKING']="1"
    # os.environ['TORCH_USE_CUDA_DSA'] = "1"

import matplotlib 
if not is_notebook():
    matplotlib.use('Agg')

# set directory
os.chdir("/nas/ucb/oliveradk/diverse-gen/")

In [2]:
# for now incomplete spurious correlation: 
#   waterbirds

# other notebooks:
# multi-class classification: 
#   multi-nli-cc

# known group labels
# waterbirds (normal)
# mulit-nli cc 


In [3]:
from tqdm import tqdm
from pathlib import Path
from datetime import datetime
from itertools import product
import json

import numpy as np

from losses.loss_types import LossType
from utils.exp_utils import get_executor, run_experiments

In [4]:
SCRIPT_NAME = "spur_corr_exp.py"
EXP_DIR = "output/cc_dbat_aux_weight_sweep"
n_trials = 32

In [10]:
method_configs = {
    "DBAT": {"loss_type": LossType.DBAT, "shared_backbone": False, "freeze_heads": True, "binary": True, "batch_size": 16, "target_batch_size": 32},
}

dataset_configs = {
    "toy_grid": {"dataset": "toy_grid", "model": "toy_model", "epochs": 100, "batch_size": 32, "target_batch_size": 128, "lr": 1e-3, "optimizer": "sgd"},
    "fmnist_mnist": {"dataset": "fmnist_mnist", "model": "Resnet50", "epochs": 5},
    "cifar_mnist": {"dataset": "cifar_mnist", "model": "Resnet50", "epochs": 5},
    "waterbirds": {"dataset": "waterbirds", "model": "Resnet50", "epochs": 5},
    "celebA-0": {"dataset": "celebA-0", "model": "Resnet50", "epochs": 2},
    "multi-nli": {"dataset": "multi-nli", "model": "bert", "epochs": 1, "lr": 1e-5, "combine_neut_entail": True, "contra_no_neg": True},
}

method_ranges = {
    "DBAT": {"aux_weight": [-1, 1]},
}

configs = {}
for (ds_name, ds_config), (method_name, method_config) in product(dataset_configs.items(), method_configs.items()):
    configs[(ds_name, method_name)] = {**ds_config, **method_config}
    if "batch_size" in ds_config:
        configs[(ds_name, method_name)]["batch_size"] = int(ds_config["batch_size"] / 2)
        configs[(ds_name, method_name)]["target_batch_size"] = int(ds_config["target_batch_size"] / 2)



def get_conf_exp_dir(ds_name, method_name, i):
    return Path(EXP_DIR, f"{ds_name}_{method_name}/{i}")

sampled_configs = []
for (ds_name, method_name), conf in configs.items():
    for i in range(n_trials):
        sample_conf = copy.deepcopy(conf)
        sample_range = method_ranges[method_name]["aux_weight"]
        aux_weight = 10**(np.random.uniform(sample_range[0], sample_range[1]))
        seed = np.random.randint(0, 1000000)
        sample_conf["aux_weight"] = aux_weight
        sample_conf["seed"] = seed
        sample_conf["exp_dir"] = get_conf_exp_dir(ds_name, method_name, i)
        sample_conf["plot_activations"] = False
        sampled_configs.append(conf)

In [ ]:
executor = get_executor(EXP_DIR, mem_gb=32)
jobs = run_experiments(sampled_configs, executor, SCRIPT_NAME)